In [1]:
# This implementation of the algorithm is based on the theory from:
# Title: QUANTUM INFORMATION AND COMPUTATION Lecture notes
# Author: Richard Jozsa
# Section: Quantum Fourier transform and periodicities
# URL: http://www.qi.damtp.cam.ac.uk/files/PartIIIQC/Part%20IIC%20QIC/PartIIC%20QIClectures%20Full.pdf

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import qiskit.quantum_info as qi
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import QFT
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_distribution, plot_histogram

In [3]:
#function f : Z_Nx -> Z_Ny with period r
a, b = 9, 2
r = 32
def f(x):
    return (a*x + b) % r

n_qx = 7
n_qy = 5
Nx = 2**n_qx
Ny = 2**n_qy

#unitary query operator for f, takes |y>|x> to |y^f(x)>|x> where ^ is bitwise XOR
U_f = np.zeros((Nx*Ny, Nx*Ny))
for x in range(Nx):
    for y in range(Ny):
        z = y ^ f(x)
        i = x + (z << n_qx)
        j = x + (y << n_qx)
        U_f[i][j] = 1

#check U_f is unitary
U_f_op = qi.Operator(U_f)
U_f_unitary = U_f_op.is_unitary()
print("U_f unitary:", U_f_unitary)
print("U_f shape:", U_f.shape)
print(U_f)

U_f unitary: True
U_f shape: (4096, 4096)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
#create quantum circuit

X = QuantumRegister(n_qx, "X")
Y = QuantumRegister(n_qy, "Y")
A = ClassicalRegister(n_qx, "A")
B = ClassicalRegister(n_qy, "B")
circuit = QuantumCircuit(X,Y,A,B)

circuit.h(X)
circuit.unitary(U_f_op, range(n_qx+n_qy), "U_f")
circuit.measure(Y, B)
circuit.barrier()
circuit.append(QFT(n_qx), range(n_qx))
circuit.measure(X, A)

display(circuit.draw())

┌───┐┌───────┐                ░ ┌──────┐┌─┐                  
X_0: ┤ H ├┤0      ├────────────────░─┤0     ├┤M├──────────────────
     ├───┤│       │                ░ │      │└╥┘┌─┐               
X_1: ┤ H ├┤1      ├────────────────░─┤1     ├─╫─┤M├───────────────
     ├───┤│       │                ░ │      │ ║ └╥┘┌─┐            
X_2: ┤ H ├┤2      ├────────────────░─┤2     ├─╫──╫─┤M├────────────
     ├───┤│       │                ░ │      │ ║  ║ └╥┘┌─┐         
X_3: ┤ H ├┤3      ├────────────────░─┤3 QFT ├─╫──╫──╫─┤M├─────────
     ├───┤│       │                ░ │      │ ║  ║  ║ └╥┘┌─┐      
X_4: ┤ H ├┤4      ├────────────────░─┤4     ├─╫──╫──╫──╫─┤M├──────
     ├───┤│       │                ░ │      │ ║  ║  ║  ║ └╥┘┌─┐   
X_5: ┤ H ├┤5      ├────────────────░─┤5     ├─╫──╫──╫──╫──╫─┤M├───
     ├───┤│   U_f │                ░ │      │ ║  ║  ║  ║  ║ └╥┘┌─┐
X_6: ┤ H ├┤6      ├────────────────░─┤6     ├─╫──╫──╫──╫──╫──╫─┤M├
     └───┘│       │┌─┐             ░ └──────┘ ║  ║  ║  ║  ║  ║ └╥┘
Y_0: ─────┤7      ├┤M├─────────────░──────────╫──╫──╫──╫──╫──╫──╫─
          │       │└╥┘┌─┐          ░          ║  ║  ║  ║  ║  ║  ║ 
Y_1: ─────┤8      ├─╫─┤M├──────────░──────────╫──╫──╫──╫──╫──╫──╫─
          │       │ ║ └╥┘┌─┐       ░          ║  ║  ║  ║  ║  ║  ║ 
Y_2: ─────┤9      ├─╫──╫─┤M├───────░──────────╫──╫──╫──╫──╫──╫──╫─
          │       │ ║  ║ └╥┘┌─┐    ░          ║  ║  ║  ║  ║  ║  ║ 
Y_3: ─────┤10     ├─╫──╫──╫─┤M├────░──────────╫──╫──╫──╫──╫──╫──╫─
          │       │ ║  ║  ║ └╥┘┌─┐ ░          ║  ║  ║  ║  ║  ║  ║ 
Y_4: ─────┤11     ├─╫──╫──╫──╫─┤M├─░──────────╫──╫──╫──╫──╫──╫──╫─
          └───────┘ ║  ║  ║  ║ └╥┘ ░          ║  ║  ║  ║  ║  ║  ║ 
A: 7/═══════════════╬══╬══╬══╬══╬═════════════╩══╩══╩══╩══╩══╩══╩═
                    ║  ║  ║  ║  ║             0  1  2  3  4  5  6 
B: 5/═══════════════╩══╩══╩══╩══╩═════════════════════════════════
                    0  1  2  3  4

In [5]:
simulator = BasicSimulator()
circuit_t = transpile(circuit, backend = simulator)

found = False
attempts = 0
while not found:
    attempts += 1
    job = simulator.run(circuit_t, shots=1)
    result = job.result()
    reg = next(iter(result.get_counts()))
    c = int(reg[-n_qx:], 2)
    q = np.gcd(c,Nx)
    r_ = Nx//q
    if (f(0) == f(r_)):
        print(f"r = {r_} found in {attempts} attempt(s).")
        found = True
        

r = 32 found in 3 attempt(s).
